<a href="https://colab.research.google.com/github/SamaSamrin/Aviation-Anomaly-Detection---Preliminary/blob/main/Real_time_anomaly_detection_in_aviation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
from google.auth import exceptions

try:
  auth.authenticate_user()
except exceptions.RefreshError as e:
  print("Authentication failed:",e)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.ensemble import IsolationForest
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score

#Display all the columns of the dataframe

pd.set_option("display.max_columns",None)

In [4]:
data_knn_2 = pd.read_csv('/content/drive/MyDrive/Masters Sept22-Aug24/0 Degree Project/March/682200107170331_fill_knn_2.csv')
data_knn_2.head()

,FPAC,BLAC,CTAC,TH,MH,EGT_1,EGT_2,EGT_3,EGT_4,IVV,GS,TRK,TRKM,DA,WS,MW,DFGS,WD,ALT,NSQT,RALT,ALTR,FQTY_1,OIT_1,OIT_2,AOA1,AOA2,PTCH,FF_1,PSA,FF_2,FF_3,ROLL,FF_4,N1_1,N1_2,MACH,CAS,APFD,PH,CASM,TAS,VRTG,LATG,PI,PS,N1_3,EVNT,MRK,VIB_1,PT,VHF1,VHF2,LGDN,LGUP,VIB_2,VHF3,PUSH,SHKR,MSQT_2,VIB_3,LONG,PLA_1,N1_4,HYDY,HYDG,VIB_4,PLA_2,PLA_3,PLA_4,GMT_HOUR,GMT_MINUTE,GMT_SEC,ACMT,FQTY_2,OIT_3,OIT_4,DATE_YEAR,DATE_MONTH,DATE_DAY,BLV,EAI,PACK,AOAI,AOAC,BAL1,BAL2,WOW,N2_1,N2_2,N2_3,N2_4,TAT,SAT,N1T,N1C,OIP_1,OIP_2,FQTY_4,CRSS,HDGS,ALTS,CASS,N1CO,VSPS,MNS,MSQT_1,VMODE,LMOD,A_T,OIP_3,OIP_4,LOC,GLS,LONP,ABRK,ECYC_1,ECYC_2,FLAP,SPLY,SPLG,FRMC,BPGR_2,BPYR_1,ECYC_3,ECYC_4,EHRS_1,DWPT,OIPL,EHRS_4,EHRS_3,EHRS_2,TMODE,ATEN,LATP,FGC3,ILSF
0,0.0,-0.011724,0.0,85.690800,98.522450,19.0,21.5,16.0,15.0,6.0,0.0,-101.516136,-88.689980,0.0,0.0,0.0,1.0,0.0,685.0,0.0,-0.625,-112.0,8024.0,26.360992,27.704071,8.217715,-14.370017,-0.340567,0.0,988.37500,0.0,0.0,0.296622,0.0,1.28125,1.75,0.0,0.0,2.0,1.0,300.0,0.0,1.003857,-0.019548,0.28125,29.199598,1.5,1.0,7.0,0.0,988.68750,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,-0.029708,-0.966795,1.5,0.0,0.0,0.0,0.17578,0.08789,-0.17578,3.0,30.0,24.0,57.0,4352.0,23.674774,25.017883,2001.0,7.0,17.0,0.0,0.0,2.0,-3.076148,0.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,15.0,14.5,90.46875,0.0,0.0,0.0,8128.0,-58.09529,-170.06714,5000.0,134.0,1.0,0.0,204.0,0.0,11.0,12.0,1.0,0.0,0.0,-0.240884,0.08502,-74.177635,119.98356,10643.0,7846.0,93.0,1.0,1.0,1997.0,24.414124,48.828247,8861.0,9280.0,10227.0,32832.0,0.0,8902.0,8498.0,7702.0,2.0,0.0,40.688553,120.0,1109093.0
1,0.0,-0.014655,0.0,86.014890,98.846535,19.0,21.5,16.0,15.0,6.0,0.0,-101.669940,-88.838295,0.0,0.0,0.0,1.0,0.0,686.0,0.0,-0.625,0.0,8032.0,27.704071,26.360992,8.217715,-14.370017,-0.340567,0.0,988.34375,0.0,0.0,0.340566,0.0,1.28125,1.75,0.0,0.0,2.0,1.0,300.0,0.0,1.013013,-0.017516,0.34375,29.199598,1.5,1.0,7.0,0.0,988.71875,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,-0.039360,-0.966795,1.5,0.0,0.0,0.0,0.17578,0.08789,-0.17578,3.0,30.0,24.0,57.0,4368.0,23.674774,25.017883,2001.0,7.0,17.0,0.0,0.0,2.0,-3.076148,0.0,1.0,7.0,0.0,0.0,0.0,0.0,0.0,15.0,14.5,90.46875,0.0,0.0,0.0,8128.0,-58.09529,-170.06714,5000.0,134.0,1.0,0.0,204.0,0.0,11.0,12.0,1.0,0.0,0.0,-0.242844,0.01716,-74.177635,119.98356,10643.0,7846.0,95.0,1.0,1.0,2001.0,19.531300,43.945423,8861.0,9280.0,10227.0,32832.0,0.0,8902.0,8498.0,7702.0,2.0,0.0,40.688553,120.0,1109093.0
2,0.0,-0.019540,0.0,86.289540,99.121185,19.0,21.5,16.0,15.0,5.0,0.0,-101.433740,-88.602090,0.0,0.0,0.0,1.0,0.0,686.0,0.0,-0.625,0.0,8024.0,26.360992,26.360992,8.217715,-14.370017,-0.351552,0.0,988.40625,0.0,0.0,0.373524,0.0,1.28125,1.75,0.0,0.0,2.0,1.0,300.0,0.0,1.010724,-0.027168,0.31250,29.200575,1.5,1.0,7.0,0.0,988.75000,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,-0.023612,-0.966795,1.5,0.0,0.0,0.0,0.17578,0.08789,-0.17578,3.0,30.0,24.0,57.0,4352.0,25.017883,25.017883,2001.0,7.0,17.0,0.0,0.0,2.0,-3.076148,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,15.0,14.5,90.46875,0.0,0.0,0.0,8128.0,-58.09529,-170.06714,5000.0,134.0,1.0,0.0,204.0,0.0,11.0,12.0,1.0,0.0,0.0,-0.107996,0.03198,-74.177635,119.98356,10643.0,7846.0,94.0,1.0,1.0,2005.0,19.531300,43.945423,8861.0,9280.0,10227.0,32832.0,0.0,8902.0,8498.0,7702.0,2.0,0.0,40.688553,120.0,1109093.0
3,0.0,-0.022471,0.0,86.580666,99.412315,19.0,21.5,16.0,15.0,3.0,0.0,-101.274445,-88.442795,0.0,0.0,0.0,1.0,0.0,685.0,0.0,-0.625,-64.0,8024.0,26.360992,26.360992,8.217715,-14.370017,-0.362538,0.0,988.43750,0.0,0.0,0.406482,0.0,1.28125,1.75,0.0,0.0,2.0,1.0,300.0,0.0,1.017591,-0.032756,0.31250,29.199598,1.5,1.0,7.0,0.0,988.75000,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,-0.034280,-0.966795,1.5,0.0,0.0,0.0,0.08789,0.17578,-0.17578,3.0,30.0,30.0,57.0,4352.0,25.017883,25.017883,2001.0,7.0,17.0,0.0,0.0,2.0,-3.076148,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,15.0,14.5,90.46875,0.0,0.0,0.0,8128.0,-58.09529,-170.06714,5000.0,134.0,1.0,0.0,204.0,0.0,11.0,12.0,1.0,0.0,0.0,-0.253232,-0.00156,-74.177635,119.98356,10643.0,7846.0,94.0,1.0,1.0,2009.0,19.531300,48.828247,8861.

In [5]:
data_knn_2.shape

(214656, 137)

In [7]:
df_knn_3 = pd.read_csv('/content/drive/MyDrive/Masters Sept22-Aug24/0 Degree Project/March/682200107170331_fill_knn_3.csv')
df_knn_3.head()

,FPAC,BLAC,CTAC,TH,MH,EGT_1,EGT_2,EGT_3,EGT_4,IVV,GS,TRK,TRKM,DA,WS,MW,DFGS,WD,ALT,NSQT,RALT,ALTR,FQTY_1,OIT_1,OIT_2,AOA1,AOA2,PTCH,FF_1,PSA,FF_2,FF_3,ROLL,FF_4,N1_1,N1_2,MACH,CAS,APFD,PH,CASM,TAS,VRTG,LATG,PI,PS,N1_3,EVNT,MRK,VIB_1,PT,VHF1,VHF2,LGDN,LGUP,VIB_2,VHF3,PUSH,SHKR,MSQT_2,VIB_3,LONG,PLA_1,N1_4,HYDY,HYDG,VIB_4,PLA_2,PLA_3,PLA_4,GMT_HOUR,GMT_MINUTE,GMT_SEC,ACMT,FQTY_2,OIT_3,OIT_4,DATE_YEAR,DATE_MONTH,DATE_DAY,BLV,EAI,PACK,AOAI,AOAC,BAL1,BAL2,WOW,N2_1,N2_2,N2_3,N2_4,TAT,SAT,N1T,N1C,OIP_1,OIP_2,FQTY_4,CRSS,HDGS,ALTS,CASS,N1CO,VSPS,MNS,MSQT_1,VMODE,LMOD,A_T,OIP_3,OIP_4,LOC,GLS,LONP,ABRK,ECYC_1,ECYC_2,FLAP,SPLY,SPLG,FRMC,BPGR_2,BPYR_1,ECYC_3,ECYC_4,EHRS_1,DWPT,OIPL,EHRS_4,EHRS_3,EHRS_2,TMODE,ATEN,LATP,FGC3,ILSF
0,0.0,-0.011724,0.0,85.690800,98.522450,19.0,21.5,16.0,15.0,6.0,0.0,-101.516136,-88.689980,0.0,0.0,0.0,1.0,0.0,685.0,0.0,-0.625,-112.0,8024.0,26.360992,27.704071,8.217715,-14.370017,-0.340567,0.0,988.37500,0.0,0.0,0.296622,0.0,1.28125,1.75,0.0,0.0,2.0,1.0,300.0,0.0,1.003857,-0.019548,0.28125,29.199598,1.5,1.0,7.0,0.0,988.68750,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,-0.029708,-0.966795,1.5,0.0,0.0,0.0,0.17578,0.08789,-0.17578,3.0,30.0,24.0,57.0,4352.0,23.674774,25.017883,2001.0,7.0,17.0,0.0,0.0,2.0,-3.076148,0.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,15.0,14.5,90.46875,0.0,0.0,0.0,8128.0,-58.09529,-170.06714,5000.0,134.0,1.0,0.0,204.0,0.0,11.0,12.0,1.0,0.0,0.0,-0.240884,0.08502,-74.177635,119.98356,10643.0,7846.0,93.0,1.0,1.0,1997.0,24.414124,48.828247,8861.0,9280.0,10227.0,32832.0,0.0,8902.0,8498.0,7702.0,2.0,0.0,40.688553,120.0,1109093.0
1,0.0,-0.014655,0.0,86.014890,98.846535,19.0,21.5,16.0,15.0,6.0,0.0,-101.669940,-88.838295,0.0,0.0,0.0,1.0,0.0,686.0,0.0,-0.625,0.0,8032.0,27.704071,26.360992,8.217715,-14.370017,-0.340567,0.0,988.34375,0.0,0.0,0.340566,0.0,1.28125,1.75,0.0,0.0,2.0,1.0,300.0,0.0,1.013013,-0.017516,0.34375,29.199598,1.5,1.0,7.0,0.0,988.71875,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,-0.039360,-0.966795,1.5,0.0,0.0,0.0,0.17578,0.08789,-0.17578,3.0,30.0,24.0,57.0,4368.0,23.674774,25.017883,2001.0,7.0,17.0,0.0,0.0,2.0,-3.076148,0.0,1.0,7.0,0.0,0.0,0.0,0.0,0.0,15.0,14.5,90.46875,0.0,0.0,0.0,8128.0,-58.09529,-170.06714,5000.0,134.0,1.0,0.0,204.0,0.0,11.0,12.0,1.0,0.0,0.0,-0.242844,0.01716,-74.177635,119.98356,10643.0,7846.0,95.0,1.0,1.0,2001.0,19.531300,43.945423,8861.0,9280.0,10227.0,32832.0,0.0,8902.0,8498.0,7702.0,2.0,0.0,40.688553,120.0,1109093.0
2,0.0,-0.019540,0.0,86.289540,99.121185,19.0,21.5,16.0,15.0,5.0,0.0,-101.433740,-88.602090,0.0,0.0,0.0,1.0,0.0,686.0,0.0,-0.625,0.0,8024.0,26.360992,26.360992,8.217715,-14.370017,-0.351552,0.0,988.40625,0.0,0.0,0.373524,0.0,1.28125,1.75,0.0,0.0,2.0,1.0,300.0,0.0,1.010724,-0.027168,0.31250,29.200575,1.5,1.0,7.0,0.0,988.75000,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,-0.023612,-0.966795,1.5,0.0,0.0,0.0,0.17578,0.08789,-0.17578,3.0,30.0,24.0,57.0,4352.0,25.017883,25.017883,2001.0,7.0,17.0,0.0,0.0,2.0,-3.076148,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,15.0,14.5,90.46875,0.0,0.0,0.0,8128.0,-58.09529,-170.06714,5000.0,134.0,1.0,0.0,204.0,0.0,11.0,12.0,1.0,0.0,0.0,-0.107996,0.03198,-74.177635,119.98356,10643.0,7846.0,94.0,1.0,1.0,2005.0,19.531300,43.945423,8861.0,9280.0,10227.0,32832.0,0.0,8902.0,8498.0,7702.0,2.0,0.0,40.688553,120.0,1109093.0
3,0.0,-0.022471,0.0,86.580666,99.412315,19.0,21.5,16.0,15.0,3.0,0.0,-101.274445,-88.442795,0.0,0.0,0.0,1.0,0.0,685.0,0.0,-0.625,-64.0,8024.0,26.360992,26.360992,8.217715,-14.370017,-0.362538,0.0,988.43750,0.0,0.0,0.406482,0.0,1.28125,1.75,0.0,0.0,2.0,1.0,300.0,0.0,1.017591,-0.032756,0.31250,29.199598,1.5,1.0,7.0,0.0,988.75000,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,-0.034280,-0.966795,1.5,0.0,0.0,0.0,0.08789,0.17578,-0.17578,3.0,30.0,30.0,57.0,4352.0,25.017883,25.017883,2001.0,7.0,17.0,0.0,0.0,2.0,-3.076148,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,15.0,14.5,90.46875,0.0,0.0,0.0,8128.0,-58.09529,-170.06714,5000.0,134.0,1.0,0.0,204.0,0.0,11.0,12.0,1.0,0.0,0.0,-0.253232,-0.00156,-74.177635,119.98356,10643.0,7846.0,94.0,1.0,1.0,2009.0,19.531300,48.828247,8861.

In [8]:
df_knn_3.shape

(214656, 137)

In [ ]:

# Function to create sliding windows
def sliding_windows(data, window_size):
    windows = []
    for i in range(len(data) - window_size + 1):
        window = data.iloc[i:i+window_size]
        windows.append(window)
    return windows

# Function to detect anomalies using Isolation Forest
def detect_anomalies(window, model):
    anomalies = model.predict(window)
    return anomalies

# Function to plot scatter graph
def plot_scatter(data, anomalies, feature1, feature2):
    plt.figure(figsize=(10, 6))
    plt.scatter(data[feature1], data[feature2], c=anomalies, cmap='viridis')
    plt.xlabel(feature1)
    plt.ylabel(feature2)
    plt.title('Anomaly Detection Scatter Plot')
    plt.colorbar(label='Anomaly Score')
    plt.show()

# Function to perform anomaly detection using Isolation Forest with sliding window
def anomaly_detection_sliding_window(data, window_size):
    # Train Isolation Forest model
    model = IsolationForest(contamination='auto', random_state=42)
    model.fit(data)

    # Create sliding windows
    windows = sliding_windows(data, window_size)

    # Detect anomalies in each window
    all_anomalies = []
    for window in windows:
        anomalies = detect_anomalies(window, model)
        all_anomalies.extend(anomalies)

    return all_anomalies

# Function to convert anomaly scores to labels based on threshold
def scores_to_labels(scores, threshold):
    labels = np.where(scores >= threshold, 1, 0)
    return labels

# Define parameters
window_size_knn_2 = int(len(data_knn_2) / 10)  # You can adjust this dynamically based on your data
window_size_knn_3 = int(len(df_knn_3) / 10)  # You can adjust this dynamically based on your data
feature1 = 'FPAC'  # Replace with your desired feature
feature2 = 'BLAC'  # Replace with your desired feature

# Perform anomaly detection with sliding window for knn_2 data
anomalies_knn_2 = anomaly_detection_sliding_window(data_knn_2, window_size_knn_2)

# Plot scatter graph for knn_2 data
plot_scatter(data_knn_2, anomalies_knn_2, feature1, feature2)

# Perform anomaly detection with sliding window for knn_3 data
anomalies_knn_3 = anomaly_detection_sliding_window(df_knn_3, window_size_knn_3)

# Plot scatter graph for knn_3 data
plot_scatter(df_knn_3, anomalies_knn_3, feature1, feature2)

# Set threshold for anomaly detection
threshold = -0.5  # Adjust this threshold as needed

# Convert anomaly scores to labels for knn_2 data
labels_knn_2 = scores_to_labels(anomalies_knn_2, threshold)

# Convert anomaly scores to labels for knn_3 data
labels_knn_3 = scores_to_labels(anomalies_knn_3, threshold)

# Evaluate the performance of the model
# Compute AUC
auc_knn_2 = roc_auc_score(labels_knn_2, anomalies_knn_2)
auc_knn_3 = roc_auc_score(labels_knn_3, anomalies_knn_3)

# Compute confusion matrix
conf_matrix_knn_2 = confusion_matrix(labels_knn_2, anomalies_knn_2)
conf_matrix_knn_3 = confusion_matrix(labels_knn_3, anomalies_knn_3)

# Compute accuracy
accuracy_knn_2 = accuracy_score(labels_knn_2, anomalies_knn_2)
accuracy_knn_3 = accuracy_score(labels_knn_3, anomalies_knn_3)

# Display results
print("Performance metrics for knn_2 data:")
print("AUC:", auc_knn_2)
print("Confusion Matrix:")
print(conf_matrix_knn_2)
print("Accuracy:", accuracy_knn_2)

print("\nPerformance metrics for knn_3 data:")
print("AUC:", auc_knn_3)
print("Confusion Matrix:")
print(conf_matrix_knn_3)
print("Accuracy:", accuracy_knn_3)


In [ ]:
# Parameter Tuning for Isolation Forest

# Function to perform anomaly detection with Isolation Forest and return AUC
def anomaly_detection_with_auc(data, contamination, random_state=42):
    model = IsolationForest(contamination=contamination, random_state=random_state)
    model.fit(data)
    anomalies = model.decision_function(data)
    labels = np.where(anomalies < 0, 1, 0)  # Thresholding anomaly scores
    auc = roc_auc_score(labels, -anomalies)  # Use negative anomalies for AUC calculation
    return auc

# Test different contamination rates
contamination_values = [0.01, 0.05, 0.1, 0.15, 0.2]

# Evaluate AUC for different contamination rates
for contamination in contamination_values:
    auc_knn_2 = anomaly_detection_with_auc(data_knn_2, contamination)
    auc_knn_3 = anomaly_detection_with_auc(df_knn_3, contamination)
    print(f"Contamination: {contamination}")
    print("AUC for knn_2 data:", auc_knn_2)
    print("AUC for knn_3 data:", auc_knn_3)
    print()


In [ ]:
# Feature Exploration

# Define a list of feature combinations to explore
feature_combinations = [
    ['FPAC', 'BLAC'],
    ['TH', 'MH'],
    ['EGT_1', 'EGT_2'],
]

# Evaluate anomaly detection performance for each feature combination
for features in feature_combinations:
    feature_data_knn_2 = data_knn_2[features]
    feature_data_knn_3 = df_knn_3[features]

    anomalies_knn_2 = anomaly_detection_sliding_window(feature_data_knn_2, window_size_knn_2)
    anomalies_knn_3 = anomaly_detection_sliding_window(feature_data_knn_3, window_size_knn_3)

    labels_knn_2 = scores_to_labels(anomalies_knn_2, threshold)
    labels_knn_3 = scores_to_labels(anomalies_knn_3, threshold)

    auc_knn_2 = roc_auc_score(labels_knn_2, anomalies_knn_2)
    auc_knn_3 = roc_auc_score(labels_knn_3, anomalies_knn_3)

    print("Feature combination:", features)
    print("AUC for knn_2 data:", auc_knn_2)
    print("AUC for knn_3 data:", auc_knn_3)
    print()


In [ ]:
from sklearn.svm import OneClassSVM

# Function to perform anomaly detection using One-Class SVM
def anomaly_detection_one_class_svm(data, window_size):
    model = OneClassSVM()
    model.fit(data)
    anomalies = model.predict(data)
    return anomalies

# Perform anomaly detection with One-Class SVM for knn_2 data
anomalies_one_class_svm_knn_2 = anomaly_detection_one_class_svm(data_knn_2, window_size_knn_2)

# Perform anomaly detection with One-Class SVM for knn_3 data
anomalies_one_class_svm_knn_3 = anomaly_detection_one_class_svm(df_knn_3, window_size_knn_3)

# Compute AUC for One-Class SVM
auc_one_class_svm_knn_2 = roc_auc_score(labels_knn_2, -anomalies_one_class_svm_knn_2)
auc_one_class_svm_knn_3 = roc_auc_score(labels_knn_3, -anomalies_one_class_svm_knn_3)

print("AUC for One-Class SVM on knn_2 data:", auc_one_class_svm_knn_2)
print("AUC for One-Class SVM on knn_3 data:", auc_one_class_svm_knn_3)


In [ ]:
from sklearn.model_selection import TimeSeriesSplit

# Perform cross-validation for Isolation Forest
tscv = TimeSeriesSplit(n_splits=5)

for train_index, test_index in tscv.split(data_knn_2):
    train_data_knn_2, test_data_knn_2 = data_knn_2.iloc[train_index], data_knn_2.iloc[test_index]
    train_anomalies_knn_2 = anomaly_detection_sliding_window(train_data_knn_2, window_size_knn_2)
    test_anomalies_knn_2 = anomaly_detection_sliding_window(test_data_knn_2, window_size_knn_2)



In [ ]:
from sklearn.metrics import classification_report

# Within the cross-validation loop
for train_index, test_index in tscv.split(data_knn_2):
    train_data_knn_2, test_data_knn_2 = data_knn_2.iloc[train_index], data_knn_2.iloc[test_index]

    # Train the model on the training data
    train_anomalies_knn_2 = anomaly_detection_sliding_window(train_data_knn_2, window_size_knn_2)

    # Evaluate the model on the test data
    test_anomalies_knn_2 = anomaly_detection_sliding_window(test_data_knn_2, window_size_knn_2)
    test_labels_knn_2 = scores_to_labels(test_anomalies_knn_2, threshold)

    # Compute performance metrics
    auc_test_knn_2 = roc_auc_score(test_labels_knn_2, -test_anomalies_knn_2)
    accuracy_test_knn_2 = accuracy_score(test_labels_knn_2, test_anomalies_knn_2)
    confusion_matrix_test_knn_2 = confusion_matrix(test_labels_knn_2, test_anomalies_knn_2)
    classification_report_test_knn_2 = classification_report(test_labels_knn_2, test_anomalies_knn_2)

    # Print or store the performance metrics
    print("Performance metrics for test set:")
    print("AUC:", auc_test_knn_2)
    print("Accuracy:", accuracy_test_knn_2)
    print("Confusion Matrix:")
    print(confusion_matrix_test_knn_2)
    print("Classification Report:")
    print(classification_report_test_knn_2)


In [ ]:
# The same process for df_knn_3

# Within the cross-validation loop
for train_index, test_index in tscv.split(df_knn_3):
    train_data_knn_3, test_data_knn_3 = df_knn_3.iloc[train_index], df_knn_3.iloc[test_index]

    # Train the model on the training data
    train_anomalies_knn_3 = anomaly_detection_sliding_window(train_data_knn_3, window_size_knn_3)

    # Evaluate the model on the test data
    test_anomalies_knn_3 = anomaly_detection_sliding_window(test_data_knn_3, window_size_knn_3)
    test_labels_knn_3 = scores_to_labels(test_anomalies_knn_3, threshold)

    # Compute performance metrics
    auc_test_knn_3 = roc_auc_score(test_labels_knn_3, -test_anomalies_knn_3)
    accuracy_test_knn_3 = accuracy_score(test_labels_knn_3, test_anomalies_knn_3)
    confusion_matrix_test_knn_3 = confusion_matrix(test_labels_knn_3, test_anomalies_knn_3)
    classification_report_test_knn_3 = classification_report(test_labels_knn_3, test_anomalies_knn_3)

    # Print or store the performance metrics
    print("Performance metrics for test set (data_knn_3):")
    print("AUC:", auc_test_knn_3)
    print("Accuracy:", accuracy_test_knn_3)
    print("Confusion Matrix:")
    print(confusion_matrix_test_knn_3)
    print("Classification Report:")
    print(classification_report_test_knn_3)
